# Weekly Project 6
Today we will continue work from monday.
We will follow the style of last week.


In [14]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt

In [15]:
def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])

d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate((0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate((d, -d, 0))

## apply k means on this
point_cloud = mesh.sample_points_uniformly(int(1e3))

- You will need to implement your own k-means algorithm. You are not allowed to use the implementation in `sklearn`.

In [16]:
def kmeans(data, K, max_attempts=500):
    """
    Simple K-means implementation with empty-cluster handling and inertia return.

    Args:
        data (np.array): An array of data points (N, D)
        K (int): The number of clusters
        max_attempts (int): max iterations for the k-means loop

    Returns:
        labels (np.array): (N,) cluster labels
        converged (bool): whether kmeans converged
        inertia (float): sum of squared distances to cluster centers
    """
    N, D = data.shape
    if N < K:
        raise ValueError("K cannot be larger than number of data points")

    rng = np.random.default_rng()
    mu = data[rng.choice(N, K, replace=False)].astype(float)

    converged = False
    attempt = 0

    labels = np.zeros(N, dtype=int)

    while not converged and attempt < max_attempts:
        distances = np.array([np.linalg.norm(data - mu[j], axis=1) for j in range(K)])
        labels = np.argmin(distances, axis=0)

        new_mu = np.zeros_like(mu)
        inertia = 0.0
        for j in range(K):
            S = data[labels == j]
            if S.shape[0] == 0:
                new_mu[j] = data[rng.integers(0, N)]
            else:
                new_mu[j] = S.mean(axis=0)
                inertia += ((S - new_mu[j]) ** 2).sum()

        if np.allclose(mu, new_mu, atol=1e-6):
            converged = True
            mu = new_mu
            break

        mu = new_mu
        attempt += 1

    distances = np.array([np.linalg.norm(data - mu[j], axis=1) for j in range(K)])
    labels = np.argmin(distances, axis=0)
    inertia = 0.0
    for j in range(K):
        S = data[labels == j]
        if S.shape[0] > 0:
            centroid = S.mean(axis=0)
            inertia += ((S - centroid) ** 2).sum()

    return labels, converged, float(inertia)

- It should be able to cluster each of the different figures.

In [17]:
mesh_model = mesh.sample_points_uniformly(int(1e3))
points = np.asarray(mesh_model.points)
labels_mesh, converge, _ = kmeans(points, 6)
if converge:
    draw_labels_on_model(mesh_model, labels_mesh)
else:
    print("The clusters did not converge")

- Extend your k-means so it finds the optimal amount of clusters.

In [18]:
def find_optimal_k(data, k_min=2, k_max=10, restarts=5):
    """
    Find optimal k by running kmeans multiple times per k and using a robust elbow detection.

    Returns:
        elbow (int), inertias (list)
    """
    inertias = []

    for k in range(k_min, k_max + 1):
        inertia_runs = []
        converged_runs = 0
        for r in range(restarts):
            _, converged, inertia = kmeans(data, k)
            if converged:
                converged_runs += 1
            inertia_runs.append(inertia)

        if converged_runs == 0:
            print(f"Warning: k={k} did not converge in any restart")
        inertias.append(float(np.median(inertia_runs)))

    inertias = np.array(inertias)

    if inertias.size < 3:
        return k_min, inertias.tolist()

    first_diff = np.diff(inertias)
    second_diff = np.diff(first_diff)

    if second_diff.size == 0:
        elbow = k_min
    else:
        elbow_idx = np.argmin(second_diff)
        elbow = k_min + elbow_idx + 1

    return int(elbow), inertias.tolist()

optimal_k, inertias = find_optimal_k(points)
print(f"Optimal k found: {optimal_k}")

Optimal k found: 6
